In [1]:
import onnx
from onnx import helper, TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.core.datatype import DataType
from qonnx.util.basic import gen_finn_dt_tensor

model = ModelWrapper("../tinyyolo_infershapes.onnx")
split_node = "BatchNormalization_10"
nodes = model.graph.node
passed_tensors=[]

for i,n in enumerate(nodes):
    if n.name != split_node:
        passed_tensors.append(n.input[1])
    else:
        s_node = n        
        break
init_tens = {}
for t in passed_tensors:
    init_tens[t] = [model.get_initializer(t),model.get_tensor_datatype(t)]
start_node = s_node.input[0]
s_node_name = s_node.name
# print(out_node)

def find_input_node(x):
    return 'x' in x.input
upstream_nodes = model.find_upstream(start_node,find_input_node)

In [2]:
up_n_ordered = []
for n in reversed(upstream_nodes):
    up_n_ordered.append(n)
ish = model.get_tensor_shape('x')
osh = model.get_tensor_shape(up_n_ordered[-1].output[0])

In [3]:
inputs = helper.make_tensor_value_info('x',TensorProto.FLOAT,ish)
outputs = helper.make_tensor_value_info(up_n_ordered[-1].output[0],TensorProto.FLOAT,osh)


In [4]:
from numpy import float32
value_info = []
for t in init_tens.items():
    shape = model.get_tensor_shape(t[0])
    value_info.append(
        model.get_tensor_valueinfo(t[0])
    )
    
new_graph = helper.make_graph(
    name ="new_graph",
    inputs=[inputs],
    outputs=[outputs],
    value_info=value_info,
    nodes=up_n_ordered
)

split_model = ModelWrapper(helper.make_model(new_graph))
for t in init_tens.keys():
    split_model.set_initializer(t,model.get_initializer(t))
    
split_model = split_model.transform(InferShapes())
split_model = split_model.transform(InferDataTypes())
model_name = "split_model_{}.onnx".format(split_node)

split_model.save(model_name)